# GSTORE CUSTOMER REVENUE PREDICTION - COMPLETE EDA
***BY HARISH JULAPALLI***

The 80/20 rule has proven true for many businesses–only a small percentage of customers produce most of the revenue. As such, marketing teams are challenged to make appropriate investments in promotional strategies.

 [](http://)[](http://) In this notebook, I've analysed GSTORE customers extensively based on various attributes. I have built visualzations to tell which specific customers made contribute to revenue of the GSTORE.

> Loading all the required libraries:

In [ ]:
import numpy as np
import pandas as pd
from pandas.io.json import json_normalize
import matplotlib.pyplot as plt
import seaborn as sns
import json

import plotly.graph_objs as go
from plotly import tools
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
init_notebook_mode(connected=True)

%matplotlib inline
plt.style.use('ggplot')

import os
print(os.listdir("../input"))

Loading the dataset:

In [ ]:
df = pd.read_csv("../input/train_v2.csv", nrows = 10000)
df.head()

As seen from a small sample of the dataset, it is evident that many columns have JSON values in them, whcih inturn have many attributes. So, we have to find a way to extract these features as well.

This function load_df helps to extract features from these JSON columns:

In [ ]:
def load_df(csv_path='../input/train_v2.csv', nrows=None):
    JSON_COLUMNS = ['device', 'geoNetwork', 'totals', 'trafficSource']
    
    df = pd.read_csv(csv_path, 
                     converters={column: json.loads for column in JSON_COLUMNS}, 
                     dtype={'fullVisitorId': 'str'}, # Important!!
                     nrows=nrows)
    
    for column in JSON_COLUMNS:
        column_as_df = json_normalize(df[column])
        column_as_df.columns = [f"{column}_{subcolumn}" for subcolumn in column_as_df.columns]
        df = df.drop(column, axis=1).merge(column_as_df, right_index=True, left_index=True)
    print(f"Loaded {os.path.basename(csv_path)}. Shape: {df.shape}")
    return df
        

Now loading train and test sets:

In [ ]:
train = load_df(nrows = 200000)
train.head()

In [ ]:
test = load_df(csv_path='../input/test_v2.csv', nrows = 50000)
test.head()

In [ ]:
train.drop(['customDimensions','hits'], axis = 1, inplace = True)
#train.head()

In [ ]:
test.drop(['customDimensions','hits'], axis = 1, inplace = True)

Remove columns which has only single category in them, as they does not add any value:

In [ ]:
single_cat_cols = [col for col in train.columns if train.nunique()[col] == 1]

In [ ]:
train.drop(single_cat_cols, axis =1, inplace = True)

In [ ]:
single_cat_cols_test = [col for col in test.columns if test.nunique()[col] == 1]
test.drop(single_cat_cols_test , axis =1, inplace = True)

In [ ]:
train.head()

## Exploratory Analysis

Number of customers who visited GSTORE:

In [ ]:
print(f"Number of unique customers: {train.fullVisitorId.nunique()}")

Number of customers who visited GSTORE more than once:

In [ ]:
print(f"Number of customers who visited the site more than once: {train.shape[0]-train.fullVisitorId.nunique()}")

Doing some conversions and filling missing values of transaction revenue with 0's:

In [ ]:
train['totals_transactionRevenue'] = train['totals_transactionRevenue'].astype('float')

In [ ]:
train['totals_transactionRevenue'].fillna(0, inplace = True)

In [ ]:
train['totals_transactionRevenue'].sum()

Isolating customers who made purchases on Gstore form the whole set:

In [ ]:
reven_group = train.groupby('fullVisitorId')['totals_transactionRevenue'].sum().reset_index()

**Plotting customers who spent money with indices on horizontal axis and log revenue on vertical axis:[[](http://)](http://)**

In [ ]:
plt.figure(figsize = (8,6))
plt.scatter(range(reven_group.shape[0]), np.sort(np.log1p(reven_group['totals_transactionRevenue'])))
plt.xlabel('Index', fontsize = 14)
plt.ylabel('Log Transaction Revenue', fontsize = 14)
plt.title("Non-zero Revenue group")
plt.show()

It can be observed that even in non-zero revenue group there are only a few customers who made large transactions. But the above plot is little obscure.

A Pie chart can show this variation very clearly.

In [ ]:
print(f"Percentage of Revenue generating customers: {(train[train['totals_transactionRevenue'] > 0].shape[0]/train.shape[0])*100}")

In [ ]:
#Pie-chart

labels = ['Non Revenue generating customers', 'Revenue generating customers']
non_revenue_perc = (train[train['totals_transactionRevenue'] == 0].shape[0]/train.shape[0])*100
revenue_perc = (train[train['totals_transactionRevenue'] > 0].shape[0]/train.shape[0])*100
sizes = [non_revenue_perc, revenue_perc]
explode = (0, 1)

fig1, ax1 = plt.subplots()
ax1.pie(sizes, explode = explode, labels = labels, autopct='%0.2f%%', colors = ['cornflowerblue','coral'], radius = 3)
plt.figure(figsize=(10,10))
ax1.axis('equal')
plt.tight_layout()
plt.show()

Only **1.08%** of total customers of Gstore actually made purchases.

**Distribution of target variable: totals_transactionRevenue**

In [ ]:
#Distribution of revenue generated
rg = train[train['totals_transactionRevenue'] > 0]['totals_transactionRevenue']

In [ ]:

plt.figure(figsize = (8,6))
sns.distplot(rg)
plt.xlabel("Total Transaction Revenue", fontsize = 14)
plt.title("Distribution of Total Transaction Revenue")
plt.show()

It is highly skewed to left side.

Applying log on target variable and plotting the distribution:

In [ ]:
plt.figure(figsize = (8,6))
sns.distplot(rg.apply(np.log1p))
plt.xlabel("Log Total Transaction Revenue", fontsize = 14)
plt.title("Distribution of Log Total Transaction Revenue")
plt.show()

Examining the skew and kurtosis if our log transformed target variable:

In [ ]:
from scipy.stats import kurtosis, skew
print(f"Skewness of transction value: {skew(rg.apply(np.log1p))}")
print(f"Kurtosis of transction value: {kurtosis(rg.apply(np.log1p))}")


Skewness and Kurtosis values indicate log of target variable is almost approximately normally distributed.

**Customers come to Gstore through various channels. Exploring which channles brings more customers:**

In [ ]:
plt.figure(figsize = (8,6))
sns.countplot(y = train['channelGrouping'], order = train['channelGrouping'].value_counts().index, palette='Blues_d')
plt.xlabel("Count", fontsize = 14)
plt.ylabel("Channel Grouping",fontsize = 14)
plt.title("Channel Grouping")
plt.show()

Seems **Organic search** and **Social channels** bring most of the customers through Gstore.

**Exploring how revenue generating customers come to Gstore via various channels:**

In [ ]:
rg = train[train['totals_transactionRevenue'] > 0]

plt.figure(figsize = (8,6))
sns.countplot(y = rg['channelGrouping'], order = rg['channelGrouping'].value_counts().index, palette='Blues_d')
plt.xlabel("Count of Revenue generating customers", fontsize = 14)
plt.ylabel("Channel Grouping",fontsize = 14)
plt.title("Channel Grouping")
plt.show()

**Though Organic search brings more customers, it is through Refferals where revenue generating customers are coming from:**

Grouping revenue generating customers based on channels and see which channels brought more revenue:

In [ ]:
channel_revenue_group = train.groupby('channelGrouping')['totals_transactionRevenue'].sum().apply(lambda x: x/10000)

plt.figure(figsize = (8,6))
sns.barplot(y = channel_revenue_group.index, x = channel_revenue_group.values,palette='Blues_d', order = channel_revenue_group.sort_values(ascending = False).index)
plt.xlabel("Total Revenue", fontsize = 14)
plt.ylabel("Channel Grouping",fontsize = 14)
plt.title("Total Revenue through various channels")
plt.show()

**More the customers, more the revenue. Referral has the most revenue of all channels**

Maximum revenue generated by various channels:

In [ ]:
train.groupby('channelGrouping')['totals_transactionRevenue'].max().apply(lambda x: x/10000).sort_values(ascending = False)

Exploring what **DEVICE BROWSER** are Gstore customers using and which brings more revenue:

Now exploring what browsers does customers use for coming to Gstore:

In [ ]:
plt.figure(figsize = (8,6))
sns.countplot(y = train['device_browser'], order = train['device_browser'].value_counts().head(8).index, palette='Blues_d')
plt.xlabel("Count", fontsize = 14)
plt.ylabel("Device Browsers",fontsize = 14)
plt.title("Device Browsers")
plt.show()

**Chrome and Safari are the top browsers that customers use to come to Gstore.**

Which browsers are our revenue generating customers using?

In [ ]:
plt.figure(figsize = (8,6))
sns.countplot(y = rg['device_browser'], order = rg['device_browser'].value_counts().index, palette='Blues_d')
plt.xlabel("Count of revenue generating customers", fontsize = 14)
plt.ylabel("Device Browser",fontsize = 14)
plt.title("Browsers of revenue generating customers")
plt.show()

Through which browser does most of the revenue come from?

In [ ]:
browser_revenue_group = rg.groupby('device_browser')['totals_transactionRevenue'].sum().apply(lambda x: x/10000)

plt.figure(figsize = (8,6))
sns.barplot(y = browser_revenue_group.index, x = browser_revenue_group.values,palette='Blues_d', order = browser_revenue_group.sort_values(ascending = False).index)
plt.xlabel("Revenue", fontsize = 14)
plt.ylabel("Device Browser",fontsize = 14)
plt.title("Revenues through various browsers")
plt.show()

**It's evident that Gstore users mostly use chrome to shop.**

What Browsers and devices do revenue generating customers use to come to Gstore?

In [ ]:
plt.figure(figsize = (8,6))
sns.countplot(y = rg['device_browser'], order = rg['device_browser'].value_counts().head(5).index, hue = rg['device_isMobile'], palette='Blues_d')
plt.xlabel("Count", fontsize = 14)
plt.ylabel("Device Browser/Device type",fontsize = 14)
plt.title("Count of Device browsers and device types of revenue generating customers")
plt.show()

It is evident that **Desktop users, using Chrome browser** spent more money than others.

What Devices do customers use to come to Gstore?

In [ ]:

plt.figure(figsize = (8,6))
sns.countplot(x = train['device_deviceCategory'], order = train.device_deviceCategory.value_counts().index, palette='Blues_d')
plt.xlabel("Device type", fontsize = 14)
plt.ylabel("Count",fontsize = 14)
plt.title("Device categories")
plt.show()

**Most of the Gstore customers use their Desktop for shopping. **

What devices does revenue generating customers use to come to Gstore?

In [ ]:
plt.figure(figsize = (8,6))
sns.countplot(x = rg['device_deviceCategory'], order = rg['device_deviceCategory'].value_counts().index, palette='Blues_d')
plt.xlabel("Device Category", fontsize = 14)
plt.ylabel("Count",fontsize = 14)
plt.title("Devices")
plt.show()

**Most Revenue generating customers use Desktop.**

What Operating systems does Gstore customers use?

In [ ]:
plt.figure(figsize = (8,6))
sns.countplot(y = train['device_operatingSystem'], order = train.device_operatingSystem.value_counts().head(8).index, palette='Blues_d')
plt.xlabel("Count", fontsize = 14)
plt.ylabel("OS",fontsize = 14)
plt.title("Operating systems")
plt.show()

**Windows and Mac are dominant OS which Gstore customers use.**

What OS are revenue generating customers usin?

In [ ]:
plt.figure(figsize = (8,6))
sns.countplot(y = rg['device_operatingSystem'], order = rg['device_operatingSystem'].value_counts().head(8).index, palette='Blues_d')
plt.xlabel("Count of revenue generating customers", fontsize = 14)
plt.ylabel("OS",fontsize = 14)
plt.title("Channel Grouping")
plt.show()

**There's a variation between above 2 plots. Above plot clearly shows users with MAC OS are buying from Gstore more rather than Windows which has more overall users.**

What is the total revenue based on different operating systems does buyers used?

In [ ]:
os_revenue_group = train.groupby('device_operatingSystem')['totals_transactionRevenue'].sum().apply(lambda x: x/10000)

plt.figure(figsize = (8,6))
sns.barplot(y = os_revenue_group.index, x = os_revenue_group.values,palette='Blues_d',order = os_revenue_group.sort_values(ascending = False).head(7).index)
plt.xlabel("Total Revenue", fontsize = 14)
plt.ylabel("OS",fontsize = 14)
plt.title("Total revenue by OS")
plt.show()

**Users with Mac spend more total money on Gstore.**

From which devices are buyers coming to Gstore?

In [ ]:
plt.figure(figsize = (8,6))
sns.countplot(y = rg['device_operatingSystem'], order = rg['device_operatingSystem'].value_counts().head(6).index, hue = rg['device_deviceCategory'], palette='Blues_d')
plt.xlabel("Count", fontsize = 14)
plt.ylabel("OS/Device",fontsize = 14)
plt.title("Customers across where devices/OS")
plt.show()

**It is interesting here to note that iOS and Android users from mobile are visting Gstore.**

**Now from which different regions customers are from?**

In [ ]:
geo_cols = [col for col in train.columns if 'geoNetwork' in col]

Where are people who visit Gstore are located on the globe?

In [ ]:
plt.figure(figsize = (8,6))
sns.countplot(y = train['geoNetwork_continent'], order = train['geoNetwork_continent'].value_counts().index)
plt.xlabel("Count", fontsize = 14)
plt.ylabel("Continenet",fontsize = 14)
plt.title("users across various continents")
plt.show()

**Most people coming to Gstore are from Americas and Asia.**

What countries are Gstore customers from?

In [ ]:
plt.figure(figsize = (14,10))
sns.countplot(x = train['geoNetwork_country'], order = train['geoNetwork_country'].value_counts().head(20).index)
plt.xticks(rotation=90)
plt.xlabel("Country", fontsize = 14)
plt.ylabel("Count",fontsize = 14)
plt.title("users across various countries")
plt.show()

Gstore sees its users mainly from **US,India,UK and Canada**.

Where are Gstore buyers mostly from?

In [ ]:
plt.figure(figsize = (8,6))
sns.countplot(y = rg['geoNetwork_country'], order = rg['geoNetwork_country'].value_counts().head(10).index, palette='Blues_d')
plt.xlabel("Country", fontsize = 14)
plt.ylabel("Count",fontsize = 14)
plt.title("revenue generating customers across various countries")
plt.show()

It's **US and Canada** where most of the revenue generating customers are from.

Since most of Gstore customers are from US, it's only logical to see from which cities these customers are from.

In [ ]:
us_traffic = train[train['geoNetwork_country'] == 'United States']

plt.figure(figsize = (10,8))
sns.countplot(y = us_traffic['geoNetwork_metro'], order = us_traffic['geoNetwork_metro'].value_counts().head(10).index)
plt.xlabel("City", fontsize = 14)
plt.ylabel("Count",fontsize = 14)
plt.title("Customers across various US cities")
plt.show()

Most of the users of Gstore are from **San Jose, New York city and Los Angeles.**

Now what does Gstore customers look on a map?

In [ ]:
tmp = train['geoNetwork_country'].value_counts()
country_visits = pd.DataFrame(data={'geoNetwork_country': tmp.values}, index=tmp.index).reset_index()
country_visits.columns = ['Country', 'Visits']

In [ ]:
colorscale = [[0, 'rgb(102,194,165)'], [0.005, 'rgb(102,194,165)'], 
              [0.01, 'rgb(171,221,164)'], [0.02, 'rgb(230,245,152)'], 
              [0.04, 'rgb(255,255,191)'], [0.05, 'rgb(254,224,139)'], 
              [0.10, 'rgb(253,174,97)'], [0.25, 'rgb(213,62,79)'], [1.0, 'rgb(158,1,66)']]

def plot_country_map(data, location, z, legend, title, colormap='Rainbow'):
    data = dict(type = 'choropleth', 
                colorscale = colorscale,
                autocolorscale = False,
                reversescale = False,
               locations = data[location],
               locationmode = 'country names',
               z = data[z], 
               text = data[z],
               colorbar = {'title':legend})
    layout = dict(title = title, 
                 geo = dict(showframe = False, 
                         projection = {'type': 'natural earth'}))
    choromap = go.Figure(data = [data], layout=layout)
    iplot(choromap)

In [ ]:
plot_country_map(country_visits, 'Country', 'Visits', 'Visits', 'Visits per country')
plt.show()

Red indicates more users and green indiactes less number or no users.

How revenue generating customers would look on a map?

In [ ]:
revenue_by_country = rg.groupby('totals_transactionRevenue')['geoNetwork_country'].sum().reset_index()

plot_country_map(revenue_by_country, 'geoNetwork_country', 'totals_transactionRevenue', 'Transaction Revenue', 'Revenue per country')
plt.show()

Red indicates more revenue as compared to whie.

Analysing the Time column might throw more light on consumer behaviour.

In [ ]:
train['date'] = pd.to_datetime(train['date'], format = '%Y%m%d')
test['date'] = pd.to_datetime(test['date'], format = '%Y%m%d')

In [ ]:
def plot_scatter_data(data, xtitle, ytitle, title, color='blue'):
    trace = go.Scatter(
        x = data.index,
        y = data.values,
        name=ytitle,
        marker=dict(
            color=color,
        ),
        mode='lines+markers'
    )
    data = [trace]
    layout = dict(title = title,
              xaxis = dict(title = xtitle), yaxis = dict(title = ytitle),
             )
    fig = dict(data=data, layout=layout)
    iplot(fig, filename='lines')

**Total visits to Gstore over the time:**

In [ ]:
count_all = train.groupby('date')['totals_transactionRevenue'].agg(['size'])
count_all.columns = ["Total"]
count_all = count_all.sort_index()
plot_scatter_data(count_all['Total'],'Date', 'Total','Total count of visits (including zero transactions)','green')

There's certainly a dip in transaction revenue during few months and a raise during others.

In [ ]:
count_nonzero = train.groupby('date')['totals_transactionRevenue'].agg(['count'])
count_nonzero.columns = ["Total"]
count_nonzero = count_nonzero.sort_index()
plot_scatter_data(count_nonzero['Total'],'Date', 'Total','Total non-zero transaction visits','darkblue')

The above interactive plot explore the buyers trend over time.

In [ ]:
total_nonzero = train.groupby('date')['totals_transactionRevenue'].agg(['sum'])
total_nonzero.columns = ["Total"]
total_nonzero = total_nonzero.sort_index()
plot_scatter_data(total_nonzero['Total'],'Date', 'Total','Total non-zero transaction amounts','red')

The above plot shows the total transaction revenue at any particular point of the time.

Now examining how the trends vary over months, days etc.

In [ ]:
train_df_date = train.set_index('date')
train_df_date['year'] = train_df_date.index.year
train_df_date['month'] = train_df_date.index.month
train_df_date['weekday'] = train_df_date.index.weekday_name
#train_df_date.head()


In [ ]:
train_df_date_rg = train_df_date[train_df_date['totals_transactionRevenue'] > 0]

In [ ]:
plt.figure(figsize = (8,6))
sns.countplot(train_df_date['year'], palette = 'Blues_d')
plt.xlabel("Year", fontsize = 14)
plt.ylabel("Count",fontsize = 14)
plt.title("Users over the years")
plt.show()

Year 2017 saw more number of visits to the Gstore.

In [ ]:
plt.figure(figsize = (8,6))
sns.countplot(train_df_date_rg['year'], palette = 'Blues_d')
plt.xlabel("Year", fontsize = 14)
plt.ylabel("Count",fontsize = 14)
plt.title("Revenue generating users over the years")
plt.show()

Year 2017, had most revenue than 2017 and 2018.

Which month bought more users?

In [ ]:
plt.figure(figsize = (8,6))
sns.countplot(train_df_date['month'], palette = 'Blues_d')
plt.xlabel("Month", fontsize = 14)
plt.ylabel("Count",fontsize = 14)
plt.title("Users over the Months")
plt.show()

November, August and February had more visitors to Gstore.

In [ ]:
plt.figure(figsize = (8,6))
sns.countplot(train_df_date_rg['month'], palette = 'Blues_d')
plt.xlabel("Month", fontsize = 14)
plt.ylabel("Count",fontsize = 14)
plt.title("Revenue generating users over the months")
plt.show()

**August bought more revenue to Gstore. Also, November and December also brought more revenue to Gstore. This may be because of the holiday season like Thanksgiving and Christmas, as most of the buyers are from US.**

In [ ]:
plt.figure(figsize = (8,6))
sns.countplot(train_df_date_rg['weekday'], palette = 'Blues_d', order = ['Monday','Tuesday','Wednesday','Thursday','Friday','Saturday','Sunday'])
plt.xlabel("Weekday", fontsize = 14)
plt.ylabel("Count",fontsize = 14)
plt.title("Users over week days")
plt.show()

Friday had more number of users in the Gstore.

In [ ]:
plt.figure(figsize = (8,6))
sns.countplot(train_df_date['weekday'], palette = 'Blues_d', order = ['Monday','Tuesday','Wednesday','Thursday','Friday','Saturday','Sunday'])
plt.xlabel("Weekday", fontsize = 14)
plt.ylabel("Count",fontsize = 14)
plt.title("Revenue generating users over week days")
plt.show()

Gstore users certainly spends more more money on Fridays and generally on Weekdays.

**Now let's further explore the adcontent, the ads/adwords which brought users to Gstore:**

In [ ]:
from wordcloud import WordCloud

ad_content = train['trafficSource_adContent'].fillna('')
wordcloud_ad = WordCloud(width=800, height=400, background_color="white").generate(' '.join(ad_content))
plt.figure( figsize=(12,9))
plt.imshow(wordcloud_ad)
plt.axis("off")
plt.show()

**Now through which sources did most users come from?**

In [ ]:
source = train['trafficSource_source'].fillna('')
wordcloud_source = WordCloud(width=800, height=400, background_color="white").generate(' '.join(source))
plt.figure( figsize=(12,9))
plt.imshow(wordcloud_source)
plt.axis("off")
plt.show()

## Conclusion

* So, based on the plots above it can be inferred that, most revenue generating customers come from US who use Chrome browser on Mac. They are most likely to spend their money on Fridays and during holiday seasons. 
* Since, we saw referrals bought more revenue than other sources, it is wise to give referral coupons or similar referral bonuses to bring more revenue to Gstore. Similar referral strategies can be used in other potential revenue generating countries like Canada, UK and India.
* From the word clouds above, it is seen that, more customers come to Gstore through google search/youtube. There is a good chance to attract customers by giving more ads in these platforms, preferably during holiday seasons of respective countries.